#Inicializando con Apache Spark

## Spark Session

La sesion de spark es la encargada de ejecutar manipulaciones definidas por el usuario en todo el clúster. Existe una correspondencia uno a uno entre una SparkSession y una aplicación Spark.

In [0]:
# Creando la sesion de spark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spark_test').getOrCreate()

In [0]:
# en Databricks, al encender nuestro cluster, este genera una sesion de spark de forma automática
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
myRange = spark.range(1000).toDF("number")


In [0]:
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



## Spark UI

La interface de spark es la que nos permitirá monitorear el progreso de un trabajo. Originalmente, la interfaz de usuario de Spark está disponible en el puerto 4040 del nodo del controlador. Si está ejecutando en modo local, será http://localhost:4040. La interfaz de usuario de Spark muestra información sobre el estado de sus trabajos de Spark, su entorno y el estado del clúster. Es muy útil, especialmente para ajustar y depurar.

<img src='https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912201/files/assets/spdg_0206.png'>

## Lectura y Escritura de Datos

Spark permite la **lectura de datos** de archivos de diferentes tipos, en forma general una lectura de datos con spark se realiza de la siguiente manera: 



<code>DataFrameReader.format(...).option("key", "value").schema(...).load()</code>


Donde:
  - Format: formato a leer
  - option: opciones de lectura
  - schema: esquema de carga (estable columna y tipo de datos de estos)
  - load: ruta a leer


Spark tambien establece los siguiente **modos de lectura**:

| Read Mode     	| Description                                                                                                                                                        	|
|---------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| permissive    	| Establece todos los campos en nulo cuando encuentra un registro corrupto  y coloca todos los registros corruptos en una columna  de cadena llamada _corrupt_record 	|
| dropMalformed 	| Elimina registros que contengan data discordante                                                                                                                   	|
| failfast      	| Falla inmediatamente al encontrar  registros con formato incorrecto                                                                                                	|

Spark ofrece la **escritura de datos** de la siguiente manera:

<code>DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(
  ...).save() </code>
  
Donde:
  - format: formato a escribir el archivo
  - option: opciones para la escritura
  - partitionBy:  particion del archivo
  - save: ruta a guardar el archivo
  
Tambien se establecen los siguiente modos de escritura:

| Write Mode     	| Description                                                                                       	|
|---------------	|---------------------------------------------------------------------------------------------------	|
| append        	| Agrega los archivos de salida a la lista de archivos  que ya existen en esa ubicación             	|
| overwrite     	| Sobrescribirá por completo cualquier dato que ya  exista en la ruta especificada                  	|
| errorIfExists 	|  Lanza un error y falla la escritura si ya existen datos  o archivos en la ubicación especificada 	|

### 1. CSV
-----------------

Archivos que contienen información separada comunmente por comas o algún otro separador

Estos archivos se leen tradicionalmente de la siguiente manera:

<code>spark.read.format("csv")</code>

In [0]:
# leeremos la data '2010-summary.csv'
df_csv = (spark.read.format("csv")
      .option("header", "true") # archivo contiene cabecera?
      .option("mode", "FAILFAST") # modo lectura
      .option("inferSchema", "true") # inferir esquema
      .load("some/path/to/file.csv")
     )

In [ ]:
# Solo en databricks permite esta función
# df_csv.display()

In [0]:
# escritura
(df_csv.write
  .format("csv") # formato escribir
  .mode("overwrite") # modo escritura
  .option("sep", "\t") # separador del archivo
  .save("/tmp/my-tsv-file.tsv") # ruta a escribir
)

### 2. Parquet
-----------------

Parquet es un almacén de datos orientado a columnas de código abierto que proporciona una variedad de optimizaciones de almacenamiento, especialmente para cargas de trabajo de análisis. 

Es un formato de archivo que funciona excepcionalmente bien con Apache Spark y, de hecho, es el formato de archivo predeterminado. 

Se recomienda escribir datos en Parquet para almacenamiento a largo plazo porque la lectura de un archivo de Parquet siempre será más eficiente que JSON o CSV. Otra ventaja de Parquet es que admite tipos complejos. Esto significa que si su columna es una matriz (que fallaría con un archivo CSV, por ejemplo), un mapa o una estructura, aún podrá leer y escribir ese archivo sin problemas. 


Aquí se explica cómo especificar Parquet como formato de lectura:

<code>spark.read.format("parquet")</code>

In [0]:
# lectura
df_parquet = (spark.read
                .format("parquet")
                .load("/data/flight-data/parquet/2010-summary.parquet")
             )

In [0]:
# escritura
df_parquet.write.format("parquet").mode("overwrite").save("/tmp/my-parquet-file.parquet")